## Gift Contract Testing

In [1]:
from web3.auto import w3
import pandas as pd
from cyberpy import generate_wallet, address_to_address
from cyberpy._message_signer import Message
from cyberpy._wallet import seed_to_privkey
from eth_account.messages import encode_defunct
import json
from time import time
import ipfshttpclient
from tqdm.notebook import tqdm
from base64 import b64encode
from multiprocess import Pool
from math import ceil
from dotenv import dotenv_values
from IPython.core.display import display, HTML

from contract_utils import instantiate_contract, execute_contract_bash, query_contract, get_ipfs_cid_from_str, get_proofs, ContractUtils


ipfs_client = ipfshttpclient.connect()

NUMBER_OF_PARTICIPANTS = 10
NUMBER_OF_ACTIVATED_PARTICIPANTS = 5
KEY_PHRASE = 'KEY PHRASE'
NICKNAME_LIST = [f'john{round(time())}{number}' for number in range(NUMBER_OF_PARTICIPANTS)]
AVATAR_CID_LIST = [get_ipfs_cid_from_str(_nickname + '_avatar') for _nickname in NICKNAME_LIST]
CLAIM_AMOUNT_LIST = [1_000_000] * NUMBER_OF_PARTICIPANTS

INITIAL_BALANCE = str(1_000_000_000)
COEF_UP = str(13)
COEF_DOWN = str(7)
TARGET_CLAIM = str(2)

WALLET_ADDRESS = dotenv_values('.env')['WALLET_ADDRESS']
WALLET_SEED = dotenv_values('.env')['WALLET_SEED']
DISPLAY_TX_EXECUTION = False
TEST_ACCOUNT_ADDRESS = 'bostrom1mxdtr8lruutugqtxgpw2sf2tl2mhzlq5fd2du0'
TEST_ACCOUNT_SEED = 'end spread blind steak install glare pride pony switch exit zone faint ' \
                    'opinion march layer illness can nest fence top debate monitor supreme noble'

INIT_SUBGRAPH_CONTRACTS = False
SUBGRAPH_CODE_ID = str(40)
NAME_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom1rncw9n73gm30vhrv6e4p603hav0gue8y5y9fgqa84k4atf5pqvfqcrnpl6'
AVATAR_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom164w2vl7z7lpuvex6z3ru0v55fgq3dmvxuqt0aejp49w7fyc8g6kshreggq'
PROOF_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom1543j9n7slzff3curyac7ylf2ctg7rk9zjf9ehj08eqx57xj33zzqdy6ga4'

INIT_PASSPORT_CONTRACT = False
PASSPORT_CODE_ID = str(43)
PASSPORT_CONTRACT_ADDRESS = 'bostrom1g59m935w4kxmtfx5hhykre7w9q497ptp66asspz76vhgarss5ensdy35s8'

INIT_GIFT_CONTRACT = True
GIFT_CODE_ID = str(20)
GIFT_CONTRACT_ADDRESS = 'bostrom1rt2acjyhs4jfjdq56pftpu7762hy9gfl63je6fnhwrc5p5y4kmuqxg0262'

ROOT_SOURCE_FILE_NAME = 'root_testing_source_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.json'
PROOF_FILE_NAME = 'proof_testing_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.json'
ROOT_FILE_NAME = 'root_testing_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses'

/Users/sergenedashkovsky/Library/Python/3.8/lib/python/site-packages/ipfshttpclient/client/__init__.py:75: VersionMismatch: Unsupported daemon version '0.10.0' (not in range: 0.5.0 ≤ … < 0.9.0)
  warnings.warn(exceptions.VersionMismatch(version, minimum, maximum))


### Generate addresses and sign messages

In [2]:
claims_list = []
for i in tqdm(range(NUMBER_OF_PARTICIPANTS)):
    claim_item = {'amount': CLAIM_AMOUNT_LIST[i],
                  'nickname': NICKNAME_LIST[i],
                  'avatar': AVATAR_CID_LIST[i]}

    # Generate Ethereum wallet
    ethereum_wallet = w3.eth.account.create(KEY_PHRASE)
    claim_item['ethereum_address'] = ethereum_wallet.address.lower()
    claim_item['ethereum_private_key'] = ethereum_wallet.privateKey.hex()

    # Generate Bostrom and Cosmos wallet
    bostrom_wallet = generate_wallet()
    claim_item['bostrom_address'] = bostrom_wallet['address']
    claim_item['cosmos_address'] = address_to_address(bostrom_wallet['address'], 'cosmos')
    claim_item['cosmos_seed'] = bostrom_wallet['seed']

    # Create message
    claim_item['message'] = f"{claim_item['bostrom_address']}:QmRX8qYgeZoYM3M5zzQaWEpVFdpin6FvVXvp6RPQK3oufV"

    # Sign message form Ethereum address
    ethereum_signed_message = \
        w3.eth.account.sign_message(
            signable_message=encode_defunct(text=claim_item['message']),
            private_key=ethereum_wallet.privateKey)
    # print(signed_message.signature)
    claim_item['ethereum_message_signature'] = ethereum_signed_message.signature.hex()

    # Sign message form Cosmos address
    cosmos_msg = Message(privkey=seed_to_privkey(seed=claim_item['cosmos_seed']))
    cosmos_msg.add_message(signing_message=claim_item['message'], signer_prefix='cosmos')
    claim_item['cosmos_message_signed_row'] = cosmos_msg.get_signed_message()
    claim_item['cosmos_message_signature'] = b64encode(json.dumps(claim_item['cosmos_message_signed_row']).replace('\n', '').replace(' ', '').encode('utf-8')).decode("utf-8")

    # Verify message
    assert claim_item['ethereum_address'] == w3.eth.account.recover_message(
        signable_message=encode_defunct(text=claim_item['message']),
        signature=claim_item['ethereum_message_signature']).lower()

    claims_list.append(claim_item)

claims_df = pd.DataFrame(claims_list)
claims_df.to_csv('claims_ethereum_test_data_without_proof_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.csv')
claims_df

  0%|          | 0/10 [00:00<?, ?it/s]

,amount,nickname,avatar,ethereum_address,ethereum_private_key,bostrom_address,cosmos_address,cosmos_seed,message,ethereum_message_signature,cosmos_message_signed_row,cosmos_message_signature
0,1000000,john16495703800,QmegFZS691C4pJFaFS3NVDNLd5PtjmMiJHJKtqzSsz2tBZ,0x9124d6592264ae9e6693dfa39b8f82f16182419c,0xcf156a52fd254b48a87c77926630089376a2e6d507f5...,bostrom1y90fdf63dz6jfahupu4wdqp6v3f69rxptnr04c,cosmos1y90fdf63dz6jfahupu4wdqp6v3f69rxpgqhutl,mixed february demise clinic master race famou...,bostrom1y90fdf63dz6jfahupu4wdqp6v3f69rxptnr04c...,0x9a2bbfc7ab3c7d4ed8910b0a495931c8a055c2fc29e7...,{'pub_key': 'A89DQdUqRt7dzpi8/1Jxyb1hh+a6Q0bGo...,eyJwdWJfa2V5IjoiQTg5RFFkVXFSdDdkenBpOC8xSnh5Yj...
1,1000000,john16495703801,QmbAqD21zVJD6D2kvoGnV1Fb8T5XKUG81V8Y7iZVHnpisC,0x042a2bb0e4acd457bcadda6e05827aa83ebe91fa,0x46a7e94a530954956533e1a923d07c212eafc00a8d23...,bostrom162f8clv5ht6j2kmms0556j0mezuj3e6ked366q,cosmos162f8clv5ht6j2kmms0556j0mezuj3e6k679fy8,quantum seed good glass spend spread usual sur...,bostrom162f8clv5ht6j2kmms0556j0mezuj3e6ked366q...,0x5297763d57331039613520d1de1c9f94b0bf104d6fd2...,{'pub_key': 'At5DOzq36uQeOGT//9kV3HcsPmy4HTAsO...,eyJwdWJfa2V5IjoiQXQ1RE96cTM2dVFlT0dULy85a1YzSG...
2,1000000,john16495703802,QmdYTELFKWGpiZ6Qaf6bDHavRMrJ5nmqMTc94mwtnNzr6M,0x0329bbdbf79eaf87d8d77ecb71f7f8d816544359,0xcdf21ac40a23b30ac2208d9e3f72c7b798068df70f39...,bostrom1he3vaq8knkx56tevl6mkel42eddq507mgulsnf,cosmos1he3vaq8knkx56tevl6mkel42eddq507mt0trdw,lava protect work pulse guess toss rocket volu...,bostrom1he3vaq8knkx56tevl6mkel42eddq507mgulsnf...,0x3b7e242a976bd013862872532234536ea2e3397d3411...,{'pub_key': 'Aph+/Mv0rOA0Ft+8iP6s7UJqTdOXK8tsK...,eyJwdWJfa2V5IjoiQXBoKy9NdjByT0EwRnQrOGlQNnM3VU...
3,1000000,john16495703803,QmTqzfWV7DsGQsyoMHJTVBRkDtKBEjy5hPErMRfpbmimPm,0x2c105e1ce5bc66a3ec0eb18089b00a471e3bd0a8,0x08669cd3a21d69b25991ac891c5d1529b64fc893f846...,bostrom1efylm4laztlkc0ydye3k6g6cx95h6khw7z27mp,cosmos1efylm4laztlkc0ydye3k6g6cx95h6khwa37d9x,medal medal pull rule blur result gift agree g...,bostrom1efylm4laztlkc0ydye3k6g6cx95h6khw7z27mp...,0x90d40af4947b8a499c22c79e1ea29cea981be1f819db...,{'pub_key': 'A055jXQh2nQUJQQdlssNe0ZWQf9KRkUba...,eyJwdWJfa2V5IjoiQTA1NWpYUWgyblFVSlFRZGxzc05lMF...
4,1000000,john16495703804,QmNWiHGs9WjmzbcQz96fugBm2Bu6CR3uZVsMuiM1tr4wYD,0xcd012b8e0c4a5bd5c9283a0a4281ec553420d149,0xb241a6f909672108710c007c1ea3173656139243deaa...,bostrom1z4gex65lzw62uqapaxctfku4588ulvgn4yvwmk,cosmos1z4gex65lzw62uqapaxctfku4588ulvgnkhca93,hope someone gown grain execute reason chaos g...,bostrom1z4gex65lzw62uqapaxctfku4588ulvgn4yvwmk...,0xcc8656d43b15788c5e0400596f5cebd760da1fd0b75e...,{'pub_key': 'Ao00rX4/WgQwgoR9wgwOTEgNq5mh9qfDI...,eyJwdWJfa2V5IjoiQW8wMHJYNC9XZ1F3Z29SOXdnd09URW...
5,1000000,john16495703805,QmUva19NDC6iVZZ9gGL7jZfsXxpPd6bijzhbUsFvSXTcbV,0xc45e14572ad2ec57da27faab28919bbda692037e,0xbac8ff0d46365a737ac95be339a9036f861bc371ce14...,bostrom13f4sr82dux0kfu7uma3atv7frhzx57ghvm9dxa,cosmos13f4sr82dux0kfu7uma3atv7frhzx57gh0g37c6,bicycle lyrics ring donate price inherit stool...,bostrom13f4sr82dux0kfu7uma3atv7frhzx57ghvm9dxa...,0xb950237f45061a45796e9f6045ef0675a32ee0df4616...,{'pub_key': 'A2Udd3YSUpIPjMuAEmdwCzFYEvHQEHxu3...,eyJwdWJfa2V5IjoiQTJVZGQzWVNVcElQak11QUVtZHdDek...
6,1000000,john16495703806,QmWXwdS2HtDFxfQ6bwdFvAjJ5HUGS7kQf3pmtkQDzYt9cd,0xb66bcd3431cc5b1999f98c608963fa9ff2a5d0d2,0x02c99ad10dc1e0b4400d72bdca2b9623975de1489887...,bostrom1pypc2s9dh0f6yp0xcqwjap99l5k75qgmp2kntw,cosmos1pypc2s9dh0f6yp0xcqwjap99l5k75qgmzezq4f,leaf try race mango beef notice opera pyramid ...,bostrom1pypc2s9dh0f6yp0xcqwjap99l5k75qgmp2kntw...,0xd767925b964cff4fb53af0c37b4b8c6b6cfdd8ca4e9a...,{'pub_key': 'AxeON8LGKhtXnzWl1yIuWli6bvIQCj9db...,eyJwdWJfa2V5IjoiQXhlT044TEdLaHRYbnpXbDF5SXVXbG...
7,1000000,john16495703807,QmS8CCPyZkNZtiyyt6KbNVKYvbv2yQHaNPEF9ghhUWUN6X,0xcb6babdbc3dd4f702237dfdd644ddbc676055e86,0x67d44f92161e85defdf10fa79c090b4a579d24ad1a60...,bostrom13ugfw2hr7s8f0yttczg8e2v6v04dlwwarsc4rw,cosmos13ugfw2hr7s8f0yttczg8e2v6v0

### Create Merkle Tree, Get Root and Proofs

In [3]:
root_source_list = [{'address': _item['ethereum_address'],
                     'amount': str(_item['amount'])} for _item in claims_list]+ \
                    [{'address': _item['cosmos_address'],
                      'amount': str(_item['amount'])} for _item in claims_list]
root_source_list.append({'address': '0xF2749114FeaAD68854E01C8eE762C7170532FdfD'.lower(), 'amount': '10000000'})
with open(ROOT_SOURCE_FILE_NAME, 'w') as outfile:
    outfile.write(str(root_source_list).replace("'", '"'))

In [4]:
NUMBER_OF_THREADS = 10
BASH_SIZE = 1000

tasks = set(
    (
        (ROOT_SOURCE_FILE_NAME,
         f'temp/proofs_{i}.json',
         i * BASH_SIZE,
         min(NUMBER_OF_PARTICIPANTS * 2 + 1, (i + 1) * BASH_SIZE + 1))
        for i in range(ceil(2 * NUMBER_OF_PARTICIPANTS/BASH_SIZE))
    )
)
print(tasks)
with Pool(processes=NUMBER_OF_THREADS) as pool:
    res = pool.starmap(get_proofs, tasks)
assert res == [True] * len(res)

{('root_testing_source_10_addresses.json', 'temp/proofs_0.json', 0, 21)}
yarn run v1.22.17
$ ts-node index.ts --input root_testing_source_10_addresses.json --output temp/proofs_0.json --start_index 0 --end_index 21
Merkle root: 3dc8421e3f4b94c8e3aed45c031e95a002007dfac135ca398314b163f15a2cee
Number of addresses in the Merkle tree: 21
Done in 3.58s.



In [5]:
roots = []
proofs_df = pd.DataFrame(columns=['address', 'amount', 'proof'])
for task in tasks:
    with open(task[1], 'r') as proof_file:
        root_and_proof_json = json.load(proof_file)
    roots.append(root_and_proof_json['merkle_root'])
    proofs_df = proofs_df.append(pd.DataFrame(root_and_proof_json['proofs']), ignore_index=True)
assert roots == [roots[0]] * len(roots)
root = roots[0]
proofs_df

,address,amount,proof
0,0x9124d6592264ae9e6693dfa39b8f82f16182419c,1000000,[78900d7c06c43ae20970087421f80e49257815363f6a3...
1,0x042a2bb0e4acd457bcadda6e05827aa83ebe91fa,1000000,[d0c8f5fc66c5a977340dd92568b1b989992f3b3c5b114...
2,0x0329bbdbf79eaf87d8d77ecb71f7f8d816544359,1000000,[9d5777f2aee55468fcd9a426466d35917086cc43855ca...
3,0x2c105e1ce5bc66a3ec0eb18089b00a471e3bd0a8,1000000,[5e42909234f9266f1a6cac6f517ca7971472cf36a9308...
4,0xcd012b8e0c4a5bd5c9283a0a4281ec553420d149,1000000,[d24b8d0cf08a253a44e55c4c134626b76f66b6b4e61e5...
5,0xc45e14572ad2ec57da27faab28919bbda692037e,1000000,[e9774d5437498124573647289f2644bf38883f338f882...
6,0xb66bcd3431cc5b1999f98c608963fa9ff2a5d0d2,1000000,[3030215ce1bd034297d5a9c5261eb17c827fdfebf7e83...
7,0xcb6babdbc3dd4f702237dfdd644ddbc676055e86,1000000,[b498a35a47dfd85c3f4e59859952eb3d24041f4deb0fc...
8,0x1b15c1f4711a8f32f299ccc6121bcac89770a13f,1000000,[2c95a091c7f94fc231386ca44cab8f7db16da40dc54cd...
9,0xcc9f8f54c6f538c6c3a8dc58aef9c591362ae0da,1000000,[82773830b6b2d0bc76832c66fc9ac3e2fb9b6e6c84fe3...


In [6]:
cosmos_proofs_df = proofs_df[proofs_df.address.str.startswith('cosmos')][['address', 'proof']]
ethereum_proofs_df = proofs_df[proofs_df.address.str.startswith('0x')][['address', 'proof']]

claims_with_proofs_df = claims_df\
                .merge(
                    ethereum_proofs_df.rename(columns={'address': 'ethereum_address', 'proof': 'ethereum_proof'}),
                    how='left',
                    on='ethereum_address')\
                .merge(
                    cosmos_proofs_df.rename(columns={'address': 'cosmos_address', 'proof': 'cosmos_proof'}),
                    how='left',
                    on='cosmos_address')

claims_with_proofs_df.to_csv('claims_ethereum_test_data_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.csv')
claims_with_proofs_df.head()

,amount,nickname,avatar,ethereum_address,ethereum_private_key,bostrom_address,cosmos_address,cosmos_seed,message,ethereum_message_signature,cosmos_message_signed_row,cosmos_message_signature,ethereum_proof,cosmos_proof
0,1000000,john16495703800,QmegFZS691C4pJFaFS3NVDNLd5PtjmMiJHJKtqzSsz2tBZ,0x9124d6592264ae9e6693dfa39b8f82f16182419c,0xcf156a52fd254b48a87c77926630089376a2e6d507f5...,bostrom1y90fdf63dz6jfahupu4wdqp6v3f69rxptnr04c,cosmos1y90fdf63dz6jfahupu4wdqp6v3f69rxpgqhutl,mixed february demise clinic master race famou...,bostrom1y90fdf63dz6jfahupu4wdqp6v3f69rxptnr04c...,0x9a2bbfc7ab3c7d4ed8910b0a495931c8a055c2fc29e7...,{'pub_key': 'A89DQdUqRt7dzpi8/1Jxyb1hh+a6Q0bGo...,eyJwdWJfa2V5IjoiQTg5RFFkVXFSdDdkenBpOC8xSnh5Yj...,[78900d7c06c43ae20970087421f80e49257815363f6a3...,[30fdf3cb4f18b08a20a2db3acc5ac961095089672200b...
1,1000000,john16495703801,QmbAqD21zVJD6D2kvoGnV1Fb8T5XKUG81V8Y7iZVHnpisC,0x042a2bb0e4acd457bcadda6e05827aa83ebe91fa,0x46a7e94a530954956533e1a923d07c212eafc00a8d23...,bostrom162f8clv5ht6j2kmms0556j0mezuj3e6ked366q,cosmos162f8clv5ht6j2kmms0556j0mezuj3e6k679fy8,quantum seed good glass spend spread usual sur...,bostrom162f8clv5ht6j2kmms0556j0mezuj3e6ked366q...,0x5297763d57331039613520d1de1c9f94b0bf104d6fd2...,{'pub_key': 'At5DOzq36uQeOGT//9kV3HcsPmy4HTAsO...,eyJwdWJfa2V5IjoiQXQ1RE96cTM2dVFlT0dULy85a1YzSG...,[d0c8f5fc66c5a977340dd92568b1b989992f3b3c5b114...,[c571aa4a3bb95f180e8865ac710746125a1d2936940a1...
2,1000000,john16495703802,QmdYTELFKWGpiZ6Qaf6bDHavRMrJ5nmqMTc94mwtnNzr6M,0x0329bbdbf79eaf87d8d77ecb71f7f8d816544359,0xcdf21ac40a23b30ac2208d9e3f72c7b798068df70f39...,bostrom1he3vaq8knkx56tevl6mkel42eddq507mgulsnf,cosmos1he3vaq8knkx56tevl6mkel42eddq507mt0trdw,lava protect work pulse guess toss rocket volu...,bostrom1he3vaq8knkx56tevl6mkel42eddq507mgulsnf...,0x3b7e242a976bd013862872532234536ea2e3397d3411...,{'pub_key': 'Aph+/Mv0rOA0Ft+8iP6s7UJqTdOXK8tsK...,eyJwdWJfa2V5IjoiQXBoKy9NdjByT0EwRnQrOGlQNnM3VU...,[9d5777f2aee55468fcd9a426466d35917086cc43855ca...,[d560e34d04482e9002c51c4155cc578ad1083221e4d87...
3,1000000,john16495703803,QmTqzfWV7DsGQsyoMHJTVBRkDtKBEjy5hPErMRfpbmimPm,0x2c105e1ce5bc66a3ec0eb18089b00a471e3bd0a8,0x08669cd3a21d69b25991ac891c5d1529b64fc893f846...,bostrom1efylm4laztlkc0ydye3k6g6cx95h6khw7z27mp,cosmos1efylm4laztlkc0ydye3k6g6cx95h6khwa37d9x,medal medal pull rule blur result gift agree g...,bostrom1efylm4laztlkc0ydye3k6g6cx95h6khw7z27mp...,0x90d40af4947b8a499c22c79e1ea29cea981be1f819db...,{'pub_key': 'A055jXQh2nQUJQQdlssNe0ZWQf9KRkUba...,eyJwdWJfa2V5IjoiQTA1NWpYUWgyblFVSlFRZGxzc05lMF...,[5e42909234f9266f1a6cac6f517ca7971472cf36a9308...,[97b75d463fd68f9b16cf586bdf6d0d6dd3044beb7a640...
4,1000000,john16495703804,QmNWiHGs9WjmzbcQz96fugBm2Bu6CR3uZVsMuiM1tr4wYD,0xcd012b8e0c4a5bd5c9283a0a4281ec553420d149,0xb241a6f909672108710c007c1ea3173656139243deaa...,bostrom1z4gex65lzw62uqapaxctfku4588ulvgn4yvwmk,cosmos1z4gex65lzw62uqapaxctfku4588ulvgnkhca93,hope someone gown grain execute reason chaos g...,bostrom1z4gex65lzw62uqapaxctfku4588ulvgn4yvwmk...,0xcc8656d43b15788c5e0400596f5cebd760da1fd0b75e...,{'pub_key': 'Ao00rX4/WgQwgoR9wgwOTEgNq5mh9qfDI...,eyJwdWJfa2V5IjoiQW8wMHJYNC9XZ1F3Z29SOXdnd09URW...,[d24b8d0cf08a253a44e55c4c134626b76f66b6b4e61e5...,[1504edb79fb0500537d30e48ee1ad4fda5c61b7ac9922...


## Instantiate Contracts
### Instantiate SUBGRAPH Contracts

In [7]:
if INIT_SUBGRAPH_CONTRACTS:
    name_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test name subgraph')
    avatar_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test avatar subgraph')
    proof_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test proof subgraph')
else:
    name_subgraph_contract_address = NAME_SUBGRAPH_CONTRACT_ADDRESS
    avatar_subgraph_contract_address = AVATAR_SUBGRAPH_CONTRACT_ADDRESS
    proof_subgraph_contract_address = PROOF_SUBGRAPH_CONTRACT_ADDRESS
print(f'Name subgraph contract address: {name_subgraph_contract_address}\n'
      f'Avatar subgraph contract address: {avatar_subgraph_contract_address}\n'
      f'Proof subgraph contract address: {proof_subgraph_contract_address}')

Name subgraph contract address: bostrom1rncw9n73gm30vhrv6e4p603hav0gue8y5y9fgqa84k4atf5pqvfqcrnpl6
Avatar subgraph contract address: bostrom164w2vl7z7lpuvex6z3ru0v55fgq3dmvxuqt0aejp49w7fyc8g6kshreggq
Proof subgraph contract address: bostrom1543j9n7slzff3curyac7ylf2ctg7rk9zjf9ehj08eqx57xj33zzqdy6ga4


### Instantiate Passport Contract

In [8]:
if INIT_PASSPORT_CONTRACT:
    passport_contract_address = \
        instantiate_contract(
            init_query=f'''{{"name":"CPT", "minter":"{WALLET_ADDRESS}", "owner":"{WALLET_ADDRESS}", "symbol":"CPT", "avatar_subgraph": "{avatar_subgraph_contract_address}", "name_subgraph": "{name_subgraph_contract_address}", "proof_subgraph": "{proof_subgraph_contract_address}"}}''',
            contract_code_id=PASSPORT_CODE_ID,
            contract_label='test passport')
else:
    passport_contract_address = PASSPORT_CONTRACT_ADDRESS
print(f'Passport contract address: {passport_contract_address}')

Passport contract address: bostrom1g59m935w4kxmtfx5hhykre7w9q497ptp66asspz76vhgarss5ensdy35s8


### Set executor in the Subgraph Contracts

In [9]:
def set_executor_subgraph(subgraph_contract_address: str, new_executor_address: str, display_data: bool = False):
    return execute_contract_bash(execute_query=f'''{{"update_executer":{{"new_executer":"{new_executor_address}"}}}}''',
                            contract_address=subgraph_contract_address,
                            gas=600000,
                            display_data=display_data)

if INIT_PASSPORT_CONTRACT or INIT_SUBGRAPH_CONTRACTS:
    set_executor_subgraph(subgraph_contract_address=name_subgraph_contract_address, new_executor_address=passport_contract_address)
    set_executor_subgraph(subgraph_contract_address=avatar_subgraph_contract_address, new_executor_address=passport_contract_address)
    set_executor_subgraph(subgraph_contract_address=proof_subgraph_contract_address, new_executor_address=passport_contract_address)

### Instantiate Gift Contract

In [10]:
if INIT_GIFT_CONTRACT:
    gift_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "passport":"{passport_contract_address}", "allowed_native":"boot", "initial_balance":"{INITIAL_BALANCE}", "coefficient_up":"{COEF_UP}", "coefficient_down":"{COEF_DOWN}", "coefficient":"{COEF_UP}", "target_claim":"{TARGET_CLAIM}"}}''',
            contract_code_id=GIFT_CODE_ID,
            amount=INITIAL_BALANCE,
            contract_label='test gift')
else:
    gift_contract_address = GIFT_CONTRACT_ADDRESS
print(f'Gift contract address: {gift_contract_address}')

Gift contract address: bostrom1nn0hd8l3fqhxhsgdeqp3rew79w2nrtm5qedrvamd7a54r2c0czjqj5hcsn


### Initiate Class of Output Parsing

In [11]:
contract_utils = ContractUtils(ipfs_client=ipfs_client,
                               address_dict= {
                                   gift_contract_address: 'Gift Contract',
                                   passport_contract_address: 'Passport Contract',
                                   WALLET_ADDRESS: 'Passport Owner Address',
                                   name_subgraph_contract_address: 'Name Subgraph Contract',
                                   avatar_subgraph_contract_address: 'Avatar Subgraph Contract',
                                   proof_subgraph_contract_address: 'Proof Subgraph Contract'})

### Register Merkle Root

In [12]:
root_register_output = execute_contract_bash(execute_query=f'''{{"register_merkle_root":{{"merkle_root":"{root}"}}}}''',
                                        from_address=WALLET_ADDRESS,
                                        contract_address=gift_contract_address)
contract_utils.parse_contract_execution_json(root_register_output)


Events

execute
	execute contract: Gift Contract

message from Passport Owner Address wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Gift Contract
	action: register_merkle_root
	merkle_root: 3dc8421e3f4b94c8e3aed45c031e95a002007dfac135ca398314b163f15a2cee
Gas used: 121,232
Tx hash: 85B31F8C7895D79C33B890F77B7F87B5F3218E217B4C489D2189575E2C3E34D4


#### Get Merkle Root form the Gift Contract

In [13]:
print(f'Gift contract {gift_contract_address}')
print(query_contract(query='''{"merkle_root": {}}''',
                     contract_address=gift_contract_address))

Gift contract bostrom1nn0hd8l3fqhxhsgdeqp3rew79w2nrtm5qedrvamd7a54r2c0czjqj5hcsn
{'data': {'merkle_root': '3dc8421e3f4b94c8e3aed45c031e95a002007dfac135ca398314b163f15a2cee'}}


### Send coins to new addresses

In [14]:
bostrom_addresses = claims_with_proofs_df.bostrom_address.to_list()
NUMBER_ADDRESSES_IN_SENDING_CHUNK = 1000
bostrom_addresses_chunks = [bostrom_addresses[i: i+ NUMBER_ADDRESSES_IN_SENDING_CHUNK] for i in range(0, len(bostrom_addresses), NUMBER_ADDRESSES_IN_SENDING_CHUNK)]

for bostrom_addresses_item in bostrom_addresses_chunks:
    send_output = contract_utils.send_coins(
            from_seed=WALLET_SEED,
            to_addresses=claims_with_proofs_df.bostrom_address.to_list(),
            amounts=[1] * len(claims_with_proofs_df.bostrom_address.to_list()),
            gas=min(71000 * len(claims_with_proofs_df.bostrom_address.to_list()), int(23e6)),
            display_data=DISPLAY_TX_EXECUTION)
    contract_utils.parse_contract_execution_json(contract_execution_json=send_output)


Events

coin received
	receiver: bostrom1y90fdf63dz6jfahupu4wdqp6v3f69rxptnr04c
	amount: 1boot
	receiver: bostrom162f8clv5ht6j2kmms0556j0mezuj3e6ked366q
	amount: 1boot
	receiver: bostrom1he3vaq8knkx56tevl6mkel42eddq507mgulsnf
	amount: 1boot
	receiver: bostrom1efylm4laztlkc0ydye3k6g6cx95h6khw7z27mp
	amount: 1boot
	receiver: bostrom1z4gex65lzw62uqapaxctfku4588ulvgn4yvwmk
	amount: 1boot
	receiver: bostrom13f4sr82dux0kfu7uma3atv7frhzx57ghvm9dxa
	amount: 1boot
	receiver: bostrom1pypc2s9dh0f6yp0xcqwjap99l5k75qgmp2kntw
	amount: 1boot
	receiver: bostrom13ugfw2hr7s8f0yttczg8e2v6v04dlwwarsc4rw
	amount: 1boot
	receiver: bostrom19pfkg9t5nvsmmjffsjv6k625xy98esgukewtyy
	amount: 1boot
	receiver: bostrom1kzrdznpsuhj8scw8p734eex07g0779vr8xw0d6
	amount: 1boot

coin spent
	spender: Passport Owner Address
	amount: 1boot
	spender: Passport Owner Address
	amount: 1boot
	spender: Passport Owner Address
	amount: 1boot
	spender: Passport Owner Address
	amount: 1boot
	spender: Passport Owner Address
	amount: 1

### Create Passports

In [15]:
for index, row in tqdm(claims_with_proofs_df[:NUMBER_OF_ACTIVATED_PARTICIPANTS].iterrows()):
    if index == 0:
        create_passport_json = contract_utils.create_passport(row, display_data=DISPLAY_TX_EXECUTION)
        contract_utils.parse_contract_execution_json(create_passport_json, row=row)
    else:
        contract_utils.create_passport(row)

0it [00:00, ?it/s]


Events

cyberlinks
	Passport Owner Address -> Nickname
	Nickname -> Passport Owner Address
	neuron: Name Subgraph Contract

	Nickname -> Avatar
	Avatar -> Nickname
	neuron: Avatar Subgraph Contract


execute
	execute contract: Passport Contract
	execute contract: Name Subgraph Contract
	execute contract: Avatar Subgraph Contract

message from bostrom1y90fdf63dz6jfahupu4wdqp6v3f69rxptnr04c wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Passport Contract
	action: mint
	minter: Passport Contract
	token_id: 1413
Gas used: 440,183
Tx hash: CDA96114D1ED58B64B4A2F38BEF6E35C4181F9CD8236B14F039BCF39D2F30C50


### Proof (Add) Address to Passports

In [16]:
for index, row in tqdm(claims_with_proofs_df[:NUMBER_OF_ACTIVATED_PARTICIPANTS].iterrows()):
    if index == 1:
        proof_ethereum_address_json = contract_utils.proof_address(row, display_data=DISPLAY_TX_EXECUTION)
        contract_utils.parse_contract_execution_json(proof_ethereum_address_json, row=row)

        proof_cosmos_address_json = contract_utils.proof_address(row, network='cosmos', display_data=DISPLAY_TX_EXECUTION)
        contract_utils.parse_contract_execution_json(proof_cosmos_address_json, row=row)
    else:
        contract_utils.proof_address(row)
        contract_utils.proof_address(row, network='cosmos')

0it [00:00, ?it/s]


Events

cyberlinks
	Nickname -> Ethereum Address
	Ethereum Address -> Nickname
	neuron: Proof Subgraph Contract


execute
	execute contract: Passport Contract
	execute contract: Proof Subgraph Contract

message from bostrom162f8clv5ht6j2kmms0556j0mezuj3e6ked366q wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Passport Contract
	action: proof_address
	nickname: john16495703801
	address: 0x042a2bb0e4acd457bcadda6e05827aa83ebe91fa
Gas used: 281,090
Tx hash: 96C7ADA679DC321B73BF5E090CFA8E086FB906E87B1133E3CD03234F4A568115

Events

cyberlinks
	Nickname -> Cosmos Address
	Cosmos Address -> Nickname
	neuron: Proof Subgraph Contract


execute
	execute contract: Passport Contract
	execute contract: Proof Subgraph Contract

message from bostrom162f8clv5ht6j2kmms0556j0mezuj3e6ked366q wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Passport Contract
	action: proof_address
	nickname: john16495703801
	address: cosmos162f8clv5ht6j2kmms0556j0mezuj3e6k679fy8
Gas

### Claim

In [17]:
for index, row in tqdm(claims_with_proofs_df[:NUMBER_OF_ACTIVATED_PARTICIPANTS].iterrows()):
    if index == 0:
        claim_ethereum_json = contract_utils.claim(row, display_data=DISPLAY_TX_EXECUTION)
        contract_utils.parse_contract_execution_json(claim_ethereum_json)

        claim_cosmos_json = contract_utils.claim(row, network='cosmos', display_data=DISPLAY_TX_EXECUTION)
        contract_utils.parse_contract_execution_json(claim_cosmos_json)
    else:
        claim_ethereum_json_2 = contract_utils.claim(row)
        contract_utils.parse_contract_execution_json(claim_ethereum_json_2)

        claim_cosmos_json_2 = contract_utils.claim(row, network='cosmos')
        contract_utils.parse_contract_execution_json(claim_cosmos_json_2)

0it [00:00, ?it/s]

{'claim': {'nickname': 'john16495703800', 'gift_claiming_address': '0x9124d6592264ae9e6693dfa39b8f82f16182419c', 'gift_amount': '1000000', 'proof': ['78900d7c06c43ae20970087421f80e49257815363f6a3a3563f5ec29d28940d1', '8cfc554eaa1feae2ff77f1307b0341fbdf7edf6be643fbe6083fd025f5cc6328', '788ddba1a2fb402628dd12efa2aafbab89e1dfbd57d5c2b279626936a990ba2b', '8fdbf3af68c6049a91547cdc34c7c7d862bc2445ba8e0688a4d334fa48e8fdf6', 'eee157a02965da2c68325270a3893ccad9644d1a717328f173758f04d9d31620']}}

Events

coin received
	receiver: bostrom1y90fdf63dz6jfahupu4wdqp6v3f69rxptnr04c
	amount: 100000boot

coin spent
	spender: Gift Contract
	amount: 100000boot

execute
	execute contract: Gift Contract

message from bostrom1y90fdf63dz6jfahupu4wdqp6v3f69rxptnr04c wasm /cosmwasm.wasm.v1.MsgExecuteContract

transfer
	recipient: bostrom1y90fdf63dz6jfahupu4wdqp6v3f69rxptnr04c
	sender: Gift Contract
	amount: 100000boot

wasm
	_contract_address: Gift Contract
	action: claim
	original: 0x9124d6592264ae9e6693dfa39b8

### Release Gift

In [18]:
for index, row in tqdm(claims_with_proofs_df[:NUMBER_OF_ACTIVATED_PARTICIPANTS].iterrows()):
    if index == 0:
        release_ethereum_json = contract_utils.release(row, display_data=DISPLAY_TX_EXECUTION)
        contract_utils.parse_contract_execution_json(release_ethereum_json)

        release_cosmos_json = contract_utils.release(row, network='cosmos', display_data=DISPLAY_TX_EXECUTION)
        contract_utils.parse_contract_execution_json(release_cosmos_json)
    else:
        contract_utils.release(row)
        contract_utils.release(row, network='cosmos')

0it [00:00, ?it/s]


Events

coin received
	receiver: bostrom1y90fdf63dz6jfahupu4wdqp6v3f69rxptnr04c
	amount: 1290000boot

coin spent
	spender: Gift Contract
	amount: 1290000boot

execute
	execute contract: Gift Contract

message from bostrom1y90fdf63dz6jfahupu4wdqp6v3f69rxptnr04c wasm /cosmwasm.wasm.v1.MsgExecuteContract

transfer
	recipient: bostrom1y90fdf63dz6jfahupu4wdqp6v3f69rxptnr04c
	sender: Gift Contract
	amount: 1290000boot

wasm
	_contract_address: Gift Contract
	action: release
	address: bostrom1y90fdf63dz6jfahupu4wdqp6v3f69rxptnr04c
	gift_address: 0x9124d6592264ae9e6693dfa39b8f82f16182419c
	stage: 9
	amount: 1,290,000
Gas used: 157,502
Tx hash: 07B209093AB4960C86A93450B0023387BA9337A3716C92F83BA92D998130DA9B

Events

coin received
	receiver: bostrom1y90fdf63dz6jfahupu4wdqp6v3f69rxptnr04c
	amount: 1290000boot

coin spent
	spender: Gift Contract
	amount: 1290000boot

execute
	execute contract: Gift Contract

message from bostrom1y90fdf63dz6jfahupu4wdqp6v3f69rxptnr04c wasm /cosmwasm.wasm.v1.MsgEx

## Passport NFT testing
### Transfer Passport

In [19]:
def get_passport_id(bostrom_address: str) -> str:
    return query_contract(query=f'''{{"tokens": {{"owner": "{bostrom_address}"}}}}''',
                          contract_address=passport_contract_address)['data']['tokens'][0]

In [20]:
for index, row in tqdm(claims_with_proofs_df[:NUMBER_OF_ACTIVATED_PARTICIPANTS].iterrows()):
    if index == 1:
        transfer_passport_json = contract_utils.transfer_passport(
            row,
            token_id=get_passport_id(row['bostrom_address']),
            to_address=WALLET_ADDRESS,
            display_data=DISPLAY_TX_EXECUTION)
        contract_utils.parse_contract_execution_json(
            transfer_passport_json,
            row=row)
    else:
        pass

0it [00:00, ?it/s]


Events

cyberlinks
	QmPcFZbajorUYkkdeowYUD1zJRAWA13N4Koy4rLeZ4cDji -> Nickname
	Nickname -> QmPcFZbajorUYkkdeowYUD1zJRAWA13N4Koy4rLeZ4cDji
	neuron: Name Subgraph Contract


execute
	execute contract: Passport Contract
	execute contract: Name Subgraph Contract
	execute contract: Avatar Subgraph Contract

message from bostrom162f8clv5ht6j2kmms0556j0mezuj3e6ked366q wasm /cosmwasm.wasm.v1.MsgExecuteContract

reply
	reply contract: Avatar Subgraph Contract

wasm
	_contract_address: Passport Contract
	action: transfer_nft
	sender: bostrom162f8clv5ht6j2kmms0556j0mezuj3e6ked366q
	recipient: Passport Owner Address
	token_id: 1414
Gas used: 385,801
Tx hash: 5F0B12D48BAB7041AACE2E9EA131AC85065B9D8AEE33D44568A4798D84785BC1


### Burn

In [21]:
for index, row in tqdm(claims_with_proofs_df[2:NUMBER_OF_ACTIVATED_PARTICIPANTS].iterrows()):
    if index == 2:
        burn_passport_json = contract_utils.burn_passport(row, token_id=get_passport_id(row['bostrom_address']), display_data=DISPLAY_TX_EXECUTION)
        contract_utils.parse_contract_execution_json(burn_passport_json, row=row)
    else:
        pass

0it [00:00, ?it/s]


Events

cyberlinks
	cyberhole -> Nickname
	Nickname -> cyberhole
	neuron: Name Subgraph Contract

	cyberhole -> Avatar
	Avatar -> cyberhole
	neuron: Avatar Subgraph Contract


execute
	execute contract: Passport Contract
	execute contract: Name Subgraph Contract
	execute contract: Avatar Subgraph Contract

message from bostrom1he3vaq8knkx56tevl6mkel42eddq507mgulsnf wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Passport Contract
	action: burn
	sender: bostrom1he3vaq8knkx56tevl6mkel42eddq507mgulsnf
	token_id: 1415
Gas used: 381,878
Tx hash: 23476318EFB8D19BDF644F8DFB81BEFF9C0CA9BB188965FEB5E7E7AE76033C5E


### Add tests for Passport contract
#### create_passport +
#### update_name
#### update_avatar
#### proof_address +
#### remove_address
#### set_minter
#### set_owner
#### set_active
#### set_subgraphs
#### transfer_nft +
#### send_nft
#### mint
#### burn +
#### approve
#### approve_all
#### revoke
#### revoke_all
#### Expirations
##### at_height
##### at_time
##### never

### Add tests for Gift contract
#### update_owner
#### update_passport_addr
#### update_target
#### register_merkle_root +
#### claim +
#### release +

### Add tests for subgraph contract
#### update_owner
#### update_executer +
